In [1]:
 conda install -c anaconda lxml 


Solving environment: done

## Package Plan ##

  environment location: /Users/devtechbox/opt/anaconda3

  added / updated specs:
    - lxml


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2022.3.29  |       hecd8cb5_1         125 KB  anaconda
    certifi-2021.10.8          |   py39hecd8cb5_2         156 KB  anaconda
    openssl-1.1.1n             |       hca72f7f_0         3.5 MB  anaconda
    ------------------------------------------------------------
                                           Total:         3.8 MB

The following packages will be SUPERSEDED by a higher-priority channel:

  ca-certificates                                 pkgs/main --> anaconda None
  certifi                                         pkgs/main --> anaconda None
  openssl                                         pkgs/main --> anaconda None



certifi-2021.10.8    | 156 KB    | #############

In [2]:
conda install -c anaconda beautifulsoup4 


Solving environment: done

# All requested packages already installed.

Retrieving notices: ...working... done

Note: you may need to restart the kernel to use updated packages.


In [3]:
 conda install -c anaconda requests 


Solving environment: done

# All requested packages already installed.

Retrieving notices: ...working... done

Note: you may need to restart the kernel to use updated packages.


In [4]:
!pip install pandas

In [6]:
from bs4 import BeautifulSoup as bs
import requests
import csv

In [7]:
url="https://satoriz-comboire.bio/collections/boissons-sans-alcools/famille-boissons-vegetales"
response = requests.get(url)
html = response.content
print(html)

b'<!doctype html>\n<html class="no-js" lang="fr">\n<head>\n  <meta charset="utf-8">\n  <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\n  <meta name="viewport" content="width=device-width,initial-scale=1">\n  <meta name="theme-color" content="#30b457">\n  <link rel="canonical" href="https://satoriz-comboire.bio/collections/boissons-sans-alcools/famille-boissons-vegetales"><link rel="shortcut icon" href="//cdn.shopify.com/s/files/1/0270/6961/2126/files/satoriz_favicon_32x32.png?v=1614295708" type="image/png"><title>Boissons sans alcools\n&ndash; Balises &quot;famille@Boissons Vegetales&quot;&ndash; Satoriz Comboire</title><!-- /snippets/social-meta-tags.liquid -->\n\n\n\n\n<meta property="og:site_name" content="Satoriz Comboire">\n<meta property="og:url" content="https://satoriz-comboire.bio/collections/boissons-sans-alcools/famille-boissons-vegetales">\n<meta property="og:title" content="Boissons sans alcools">\n<meta property="og:type" content="product.group">\n<meta pr

In [8]:
soup = bs(html, "lxml")
print(soup)

<!DOCTYPE html>
<html class="no-js" lang="fr">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width,initial-scale=1" name="viewport"/>
<meta content="#30b457" name="theme-color"/>
<link href="https://satoriz-comboire.bio/collections/boissons-sans-alcools/famille-boissons-vegetales" rel="canonical"/><link href="//cdn.shopify.com/s/files/1/0270/6961/2126/files/satoriz_favicon_32x32.png?v=1614295708" rel="shortcut icon" type="image/png"/><title>Boissons sans alcools
– Balises "famille@Boissons Vegetales"– Satoriz Comboire</title><!-- /snippets/social-meta-tags.liquid -->
<meta content="Satoriz Comboire" property="og:site_name"/>
<meta content="https://satoriz-comboire.bio/collections/boissons-sans-alcools/famille-boissons-vegetales" property="og:url"/>
<meta content="Boissons sans alcools" property="og:title"/>
<meta content="product.group" property="og:type"/>
<meta content="Faites vos courses bio en ligne sur le

In [9]:
pages_number= int(soup.find("li", class_="pagination__text").get_text().replace("\n", "").split(' ')[7])
print(pages_number)


2


In [10]:
i=1
urlList=[]
while i <= pages_number:
    urlPage="https://satoriz-comboire.bio/collections/boissons-sans-alcools/famille-boissons-vegetales?page="+str(i)
    print(urlPage)
    response = requests.get(urlPage)
    html = response.content
    soup = bs(html, "lxml")
    urls=soup.find_all("a", class_="grid-view-item__link")
    for url in urls:
        urlList.append('https://satoriz-comboire.bio'+url['href'])
    i+= 1
    print(urls)

https://satoriz-comboire.bio/collections/boissons-sans-alcools/famille-boissons-vegetales?page=1
[<a class="grid-view-item__link grid-view-item__image-container full-width-link" href="/collections/boissons-sans-alcools/products/in1">
<span class="visually-hidden">Boisson Riz Nature Lt</span>
</a>, <a class="grid-view-item__link grid-view-item__image-container full-width-link" href="/collections/boissons-sans-alcools/products/lbt235">
<span class="visually-hidden">Boisson Amande Lt</span>
</a>, <a class="grid-view-item__link grid-view-item__image-container full-width-link" href="/collections/boissons-sans-alcools/products/lbt211">
<span class="visually-hidden">Boisson Riz Lithothamne Lt</span>
</a>, <a class="grid-view-item__link grid-view-item__image-container full-width-link" href="/collections/boissons-sans-alcools/products/lbt259">
<span class="visually-hidden">Boisson Avoine Lt</span>
</a>, <a class="grid-view-item__link grid-view-item__image-container full-width-link" href="/colle

In [11]:


def product_obj(url):
    dataObj = [];
    response = requests.get(url)
    html = response.content
    soup = bs(html, "lxml")
    dataObj.append(url) ;
    dataObj.append(soup.find("h1", class_="product-single__title").get_text())
    dataObj.append(soup.find("div", class_="product__vendor").get_text().split("\n")[1].split("  ")[6])
    dataObj.append(soup.find("span", class_="rqp").get_text().split("/")[0])
    dataObj.append(soup.find("img", class_="feature-row__image")["src"] if soup.find("img", class_="feature-row__image") else "")
    dataObj.append(soup.find("div", class_="product__refs").get_text().split(": ")[2].split("\n")[0])
    return dataObj

headers=["product Url","product Title", "product Brand","product Price", "product Image Url","product Ref"]
scrapedData=[]
for product_url in urlList:
    scrapedData.append(product_obj(product_url))
    
print(scrapedData)


[['https://satoriz-comboire.bio/collections/boissons-sans-alcools/products/in1', 'Boisson Riz Nature Lt', 'Le bio pour tous', '€1,40', '//cdn.shopify.com/s/files/1/0270/6961/2126/products/e24e00a882d400424a80158adb394430_300x300.jpg?v=1664754253', 'IN1'], ['https://satoriz-comboire.bio/collections/boissons-sans-alcools/products/lbt235', 'Boisson Amande Lt', 'Le bio pour tous', '€2,65', '//cdn.shopify.com/s/files/1/0270/6961/2126/products/079278bc63c02e75b10b17775fee0ab8_300x300.jpg?v=1664754139', 'LBT235'], ['https://satoriz-comboire.bio/collections/boissons-sans-alcools/products/lbt211', 'Boisson Riz Lithothamne Lt', 'Le bio pour tous', '€1,50', '//cdn.shopify.com/s/files/1/0270/6961/2126/products/2d0dffd2f4fbe859673f4368b6c8c7c4_300x300.jpg?v=1664754242', 'LBT211'], ['https://satoriz-comboire.bio/collections/boissons-sans-alcools/products/lbt259', 'Boisson Avoine Lt', 'Le bio pour tous', '€1,50', '//cdn.shopify.com/s/files/1/0270/6961/2126/products/5b0d1c1822b8130c1b02046f61db21d0_30

In [12]:
print(len(scrapedData))
with open('scrappedData.csv', 'w') as f:
    # using csv.writer method from CSV package
    write = csv.writer(f)
      
    write.writerow(headers)
    write.writerows(scrapedData)


63
